<a href="https://colab.research.google.com/github/smnststn/Fruits_vegetables/blob/main/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
import os
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Input,
    Dropout
)

from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.applications import MobileNetV3Large
from keras.layers import GlobalAveragePooling2D
from keras.models import Model

In [ ]:
def plot_history(history):
  """
  Funkce na vykreslení dat z objectu history -> vrací ho metoda fit keras modelu
  """
  # plot the Loss curve(s)
  plt.figure(figsize=[8,6])

  plt.plot(history.history['loss'],'r',linewidth=3.0)
  plt.plot(history.history['val_loss'],'b',linewidth=3.0)

  plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
  plt.xlabel('Epochs ',fontsize=16)
  plt.ylabel('Loss',fontsize=16)
  plt.title('Loss Curves',fontsize=16)
  plt.show()

  # plot the Accuracy curve(s)
  plt.figure(figsize=[8,6])

  plt.plot(history.history['accuracy'], 'r', linewidth=3.0)
  plt.plot(history.history['val_accuracy'], 'b',linewidth=3.0)

  plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
  plt.xlabel('Epochs ',fontsize=16)
  plt.ylabel('Accuracy',fontsize=16)
  plt.title('Accuracy Curves',fontsize=16)
  plt.show()

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kritikseth/fruit-and-vegetable-image-recognition")

print("Path to dataset files:", path)

100%|██████████| 1.98G/1.98G [00:23<00:00, 89.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kritikseth/fruit-and-vegetable-image-recognition/versions/8


**KOMENTÁR Šimon:**

* pridal som funkciu load images, v ktorej som využil kód, na ktorom sme
pracovali v prvý deň. Túto funkciu použijeme pre každý folder (train/test/validation)
* trochu som premenoval niektoré premenné do angličtiny a v komentároch vysvetlil, ten komentár tám môžeme nechať aj pre prehľadnosť k obhajobe
* podarilo sa mi nájsť zjednodušenie prevodu grayscale/RGBA na RGB
* snažil som sa o prehľadnosť a úspornosť kódu

**Otázka na zváženie:**
- Chceme mať radšej premenné a názvy funkcií radšej v češtine, napr. namiesto *load_images --> načíst_data*

In [ ]:
def load_images(path, folder, image_width = 224, image_height = 224):
  '''Function returns  X=images, y=labels, encoding=labels for each category'''

  X = []
  y = []
  encoding = {}

  '''
  path = path to dataset
  folder = train, validation or test
  category = vegetable or fruit
  filename = name of image

  path to image = path + folder + category + filename
             e.g. path / train / beetroot / Image_1.jpg
  '''

  folder_path = os.path.join(path, folder)

  # Loop for each category (fruit/vegetable)
  for index, category in enumerate(os.listdir(folder_path)[0:7]):

    encoding[index] = category
    category_path = os.path.join(folder_path, category)
    image_list = []

    # Loop for each file
    for filename in os.listdir(category_path):

      # Including all formats
      if filename.endswith((".jpg",".png",".jpeg")):
        img_path = os.path.join(category_path, filename)
        img = Image.open(img_path)

        # Converts to RGB (in case image mode is RGBA = with transparent background)
        img.convert('RGB')

        # Resizes to requested width/height (default 224x224)
        img_resized = img.resize((image_width, image_height))
        img_array = np.array(img_resized)
        image_list.append(img_array)

    X += image_list
    y += [index] * len(image_list)

  return X, y, encoding

In [ ]:
X_test, y_test, encoding_test = load_images(path, 'test')
X_train, y_train, encoding_train = load_images(path, 'train')
X_val, y_val, encoding_val = load_images(path, 'validation')

In [ ]:
X_test[0]

In [ ]:
encoding_test[y_test[0]]

**Kontrola, či indexy a priradené kategórie (názvy ovocia/zeleniny) sedia**

In [ ]:
# Check if indexes are identical across all encodings
if set(encoding_test.keys()) != set(encoding_train.keys()) or set(encoding_test.keys()) != set(encoding_val.keys()):
  print("(!) Inconsistent indexes across encodings")
else: print("Index are consistent")

# Check if labels are identical across all encodings.
for key in encoding_test.keys():
  if encoding_test[key] != encoding_train[key] or encoding_test[key] != encoding_val[key]:
    print(f"(!) Inconsistent label for index {key}: \n test: {encoding_test[key]},\n train: {encoding_train[key]},\n val: {encoding_val[key]}")
  else: print(f"Labels for index {key} are consistent")

**toto už je pôvodný nezmenený kód**

In [ ]:
slozky_ovoce_zelenina = os.listdir(path)
slozky_ovoce_zelenina

['test', 'validation', 'train']

In [ ]:
train_dir = os.path.join(path, 'train')
validation_dir = os.path.join(path, 'validation')
test_dir = os.path.join(path, 'test')

In [ ]:
# potřebuji načíst fotky, k fotkám labely a uložit si, jaký label patří k jaké figurce
X = []
y = []
encoding = {}
# pro každou složku s ovocem a zeleniou
for index, ovoce_zelenina in enumerate(os.listdir(train_dir)):
  # uložíme si přiřazení číel k labelům y
  encoding[index] = ovoce_zelenina
  # složka s fotkami kontrétní ovoce a zeleniny
  folder_path = os.path.join(train_dir, ovoce_zelenina)

  # sem načteme fotky konkrétní ovoce a zeleniny
  image_list = []
  # pro každý soubor ze složky
  for filename in os.listdir(folder_path):
    # pokud se jedná o .jpg
    if filename.endswith((".jpg",".png",".jpeg")):
        # cesta ke kokrétnímu souboru
        img_path = os.path.join(folder_path, filename)
        # načti obrázek ze souboru
        img = Image.open(img_path)
        # Změň rozlišení na 224x224 -> všechny budou mít stejný rozměr
        img_resized = img.resize((224, 224))
        # převeď na numpy array
        img_array = np.array(img_resized)
         # Check if the image has 3 channels (RGB)
        if img_array.ndim == 2:  # If grayscale, convert to RGB
            img_array = np.stack((img_array,) * 3, axis=-1) # Convert to 3 channels if needed
            # Check if the image has 4 channels (RGBA) and convert if necessary
        elif img_array.shape[2] == 4:  # If RGBA, convert to RGB
            img_array = img_array[:,:,:3] # Removing the alpha channel by selecting only the first 3 (RGB) channels.
        # přidat do listu k ostatním
        image_list.append(img_array)
  # do X přidej obrázky z další složky (respektive ovoce a zelenina)
  X += image_list
  # vytvoříme list s labely (rovnou jako čísla, ať nemusíme pozdějí dělat encoding)
  # přiřazení čísel k labelům máme v proměnné 'encoding'
  y += [index] * len(image_list)

In [ ]:
# převést list s numpy arrayi pro každou figurku na numpy array (tenzor)
X = np.stack(X)
# převést list y na numpy array
y = np.array(y)

In [ ]:
# transfer learning viz. níže -> zvolili jsme Mobilnetv3 jako základ
# ten vyžaduje hodnoty pixelů v rozmezí od 0-255 -> neškálovat!
X_not_scaled = X.copy()

In [ ]:
# škálujeme pro náš vlastní model
X = X / 255

In [ ]:
# rozdělení, trénink test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)
# one hot encoding y -> musí sedět rozměr s výstupem softmax vrstvy
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
early = EarlyStopping(
    # sleduj přesnost u validačních dat
    monitor="val_accuracy",
    # pokud se nezlepší alespoň o 0.3%
    min_delta=0.003,
    # za posledních 10 epoch
    patience=10,
    # tak to vypni, a načti váhy z epochy, kdy byla nejvyšší přesnost na validačních datech
    restore_best_weights=True,
    start_from_epoch = 50
)

save = ModelCheckpoint(
    # kam uložit model
    filepath="best_model.keras",
    # sleduj přesnost u validačních dat
    monitor="val_accuracy",
    # ulož vždy jen jeden soubor s nejlepší hodnotou metriky
    save_best_only=True,
    # ulož do jednoho souboru architekturu i váhy
    save_weights_only=False,
    # po každé epoše
    save_freq="epoch"
)